In [9]:
# type: ignore

import openai

# list models
models = openai.Model.list()

# print the first model's id
print(models.data[0].id) 

# create a chat completion
chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo-0613", messages=[{"role": "user", "content": "Hello world"}])

# print the chat completion
print(chat_completion.choices[0].message.content)

whisper-1
Hello! How can I assist you today?


In [10]:
import openai
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


In [18]:
import psycopg
import os

db_connection_string = os.environ["DATABASE_URL"]
conn = psycopg.connect(db_connection_string)
cur = conn.cursor()

cur.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'Agents' AND column_name = 'agent_type';")
fone = cur.fetchone()
print(fone)

None


In [19]:
cur.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'Agents';")
fone = cur.fetchall()
print(fone)

[]


In [14]:
# type: ignore

# Step 1: send the conversation and available functions to GPT
messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
    n=3,  # return 3 completions
)

response_message = response["choices"][0]["message"]


In [15]:
response

<OpenAIObject chat.completion id=chatcmpl-7UTWIXWmAQCCfEG176xPXA2c1V4Pl at 0x129794f37d0> JSON: {
  "id": "chatcmpl-7UTWIXWmAQCCfEG176xPXA2c1V4Pl",
  "object": "chat.completion",
  "created": 1687497670,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    },
    {
      "index": 1,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    },
    {
      "index": 2,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",

In [ ]:
# type: ignore


def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response


print(run_conversation())

In [6]:
# type: ignore
models.data


[<Model model id=whisper-1 at 0x1297865aa50> JSON: {
   "id": "whisper-1",
   "object": "model",
   "created": 1677532384,
   "owned_by": "openai-internal",
   "permission": [
     {
       "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
       "object": "model_permission",
       "created": 1683912666,
       "allow_create_engine": false,
       "allow_sampling": true,
       "allow_logprobs": true,
       "allow_search_indices": false,
       "allow_view": true,
       "allow_fine_tuning": false,
       "organization": "*",
       "group": null,
       "is_blocking": false
     }
   ],
   "root": "whisper-1",
   "parent": null
 },
 <Model model id=babbage at 0x1297865ab10> JSON: {
   "id": "babbage",
   "object": "model",
   "created": 1649358449,
   "owned_by": "openai",
   "permission": [
     {
       "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
       "object": "model_permission",
       "created": 1669085501,
       "allow_create_engine": false,
       "allow_sampling": true,
       

In [2]:
chat_completion



<OpenAIObject chat.completion id=chatcmpl-7UTHbxFbo6JbgCX1a8dPvkK1Dpk2f at 0x12978659130> JSON: {
  "id": "chatcmpl-7UTHbxFbo6JbgCX1a8dPvkK1Dpk2f",
  "object": "chat.completion",
  "created": 1687496759,
  "model": "gpt-3.5-turbo-0301",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 10,
    "completion_tokens": 9,
    "total_tokens": 19
  }
}